In [ ]:
from itertools import combinations

import dgl

In [ ]:
all_authors = set()

with open('dataset/paper_author_relationship.csv') as f:
    for paper, authors in enumerate(f):
        all_authors.update(map(int, authors.split(',')))

    author_mapping = {k: v for k, v in zip(all_authors, range(len(all_authors)))}
    num_authors = len(all_authors)

    f.seek(0)

    edges = set()
    for paper, authors in enumerate(f):
        authors = map(int, authors.split(','))
        authors = sorted(author_mapping[author] for author in authors)
        clique = combinations(authors, r=2)
        edges.update(clique)
    graph = dgl.to_bidirected(dgl.graph(tuple(zip(*edges))))

dgl.save_graphs('dataset/paper_author_relationship.bin', [graph])
